1. 데이터 전처리 단계 : 텍스트를 모델이 이해할 수 있는 숫자로 변환
    - `토큰화 & 정수 인코딩` : 문장을 단어와 같은 작은 단위(토큰)로 나누고, 각 단어에 고유한 숫자(정수)를 부여하는 과정
    - `워드 임베딩` : 정수 인코딩된 단어들을 의미를 함축한 고차원의 실수 벡터로 변환하는 기술. 모델은 단어간의 의미적 유사성을 학습
    - `시퀀스 패딩` : 딥러닝 모델에 입력하려면 모든 문장의 길이를 동일하게 맞춰야함. 문장 길이를 맞추기 위해 특정 숫자 (보통0)을 채워 넣어 문장의 길이를 통일

2. 딥러닝 모델링 단계 : 텍스트의 특징을 학습하여 감성 분류
    - `Baseline` : 일반 신경망 (Dense) : 워드 임베딩 벡터를 단순히 펼쳐서 입력으로 사용
        - 장점: 구현이 간단하여 기준 성능(Baseline)을 확인하기 좋음
        - 단점: 단어의 순서나 문맥을 고려하지 못해 성능에 한계
    - `Simple RNN(Recurrent Neural Network)` :  순환 신경망으로, 단어의 순서를 고려하여 문맥을 파악할 수 있는 모델. 전 단어의 정보를 다음 단어를 처리할 때 함께 사용
    - `LSTM (Long Short-Term Memory)` : Simple RNN의 기울기 소실 문제 해결. 장기적인 정보도 기억
    - `Bidirectional LSTM (양방향 LSTM)` : LSTM을 개선. 문장을 앞에서 뒤로 그리고 뒤에서 앞으로 양방향으로 읽어 문맥을 더 정확하게 파악

* 토큰화 : 텍스트를 숫자로 변환하는 과정
- 토큰화 3단계
    - 1. fit_on_text(texts) -> 가장 빈도가 높은 단어의 인덱스를 구축해서 딕셔너리 생성
    - 2. texts_to_sequence(texts) -> 각 문서를 정수 시퀀스 변환
    - 3. pad_sequence() -> 길이 정규화 (같은 길이)

In [ ]:
# 딥러닝에서 워드 임베딩 레이어 : 각 단어를 고정된 크기의 실수 벡터

In [3]:
sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

In [4]:
# 파이토치 버전
# 토큰화

from collections import Counter
import numpy as np
import torch

In [ ]:
# 단어 분할 및 빈도 계산
all_words = []
for i in [review.split() for review in sample_reviews]:
    all_words.extend(i)

In [ ]:
# 단어빈도
word_freq = Counter(all_words)
word_freq.most_common(2)        # 상위 2개

[('movie', 3), ('great', 2)]

In [ ]:
# 1. tokenizer 구현
class SimpleTokenizer:
    def __init__(self,num_words = 10, oov_token = 'UNK'):       # 토크나이저 초기화/ num_words: 토크나이저가 단어장에 포함할 최대 단어 수/ oov_token('Out-Of-Vocabulary') 단어장에 없는 단어를 대체할 특별 토큰. (기본값: 'UNK')
        self.num_words = num_words
        self.oov_token = oov_token
        self.word_index = {}
        self.index_word = {}
    def fit_on_texts(self,texts):       # 단어장 생성
        all_words = []
        for i in [review.split() for review in sample_reviews]:
            all_words.extend(i)
        word_freq = Counter(all_words)
        # 빈도 높은 순서로 인덱스 부여
        # oov 토큰을 1로 설정
        self.word_index[self.oov_token] = 1
        self.index_word[1] = self.oov_token
        idx = 2      
        for word, _ in word_freq.most_common(self.num_words -1):
            self.word_index[word] = idx
            self.index_word[idx] = word
            idx += 1
    def texts_to_sequences(self,texts):     # 텍스트를 정수 시퀀스로 변환
        ''' 텍스트를 정수 시퀀스로 변환'''
        sequences = []
        for text in texts:
            seq = []
            for word in text.split():
                # 단어가 vocabulary에 있으면 인덱스를 사용하고 없으면 oov
                word_index = self.word_index.get(word,1)
                seq.append(word_index)
            sequences.append(seq)
        return sequences
    
# tokenizer 생성 및 학습
tokenizer = SimpleTokenizer(num_words=10, oov_token='UNK')  
tokenizer.fit_on_texts(sample_reviews)
tokenizer.word_index


{'UNK': 1,
 'movie': 2,
 'great': 3,
 'wonderful': 4,
 'this': 5,
 'is': 6,
 'and': 7,
 'bad': 8,
 'with': 9,
 'poor': 10}

In [17]:
# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sample_reviews)
sequences

[[5, 2, 6, 3, 7, 4], [8, 2, 9, 10, 1], [3, 2, 1, 4]]

In [23]:
# 패딩 구현 - 문자열의 길이를 동일하게 맞춘다
def pad_sequence_manual(sequence, max_len = 10, padding='pre',value = 0):
    '''패딩구현'''
    # 1단계 : 결과물을 담을 빈 리스트 생성
    padded = []
    # 2단계 : 입력된 모든 시퀀스에 대해 반복 작업 수행
    for seq in sequences :
        # 3단계 : 현재 시퀀스의 길이가 최대길이보다 긴지 짧은지 확인
        if len(seq) >= max_len:
            if padded == 'pre':
                padded_seq = seq[-max_len:]
            else:
                padded_seq = seq[:max_len]
        else:
            pad_length = max_len-len(seq)
            if padding == 'pre':
                padded_seq = [value]*pad_length + seq
            else:
                padded_seq = seq + [value]*pad_length
        padded.append(padded_seq)
    return np.array(padded)
padded = pad_sequence_manual(sequences)
padded


array([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
       [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
       [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [20]:
# pytorch tensor 변환
sequence_tensor = torch.LongTensor(padded)

In [ ]:
sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

In [25]:
# 2. 워드 임베딩
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.nn as nn
print(f'패딩된 시퀀스 형태 : {sequence_tensor.shape}')
print(f'첫번째 : {sequence_tensor[0]}')
# pytorch embedding 레이어 생성
# num_embeddings 어휘의 크기
# embedding_dim 각 단어를 몇차원 벡터로 표현할 건지
# padding_idx 길이 맞출 때 채우는 값
embedding_layer = nn.Embedding(num_embeddings=1000, embedding_dim=8, padding_idx=0)
embedded = embedding_layer(sequence_tensor)
print(f'입력형태 : {sequence_tensor.shape}')
print(f'출력형태 : {embedded.shape}')

패딩된 시퀀스 형태 : torch.Size([3, 10])
첫번째 : tensor([0, 0, 0, 0, 5, 2, 6, 3, 7, 4])
입력형태 : torch.Size([3, 10])
출력형태 : torch.Size([3, 10, 8])


In [27]:
# 임베딩 벡터 상세 분석
# 샘플데이터의 첫 3개 단어 임베딩
for word_idx in range(3):
    embedded_vec = embedded[0,word_idx].detach().numpy()
    word_id = sequence_tensor[0,word_idx].item()
    print(f'단어 id { word_id} : {embedded_vec[:4]}--8차원 중 처음 4개')

단어 id 0 : [0. 0. 0. 0.]--8차원 중 처음 4개
단어 id 0 : [0. 0. 0. 0.]--8차원 중 처음 4개
단어 id 0 : [0. 0. 0. 0.]--8차원 중 처음 4개


In [30]:
# 임베딩 행렬
embedding_matrix = embedding_layer.weight.detach().numpy()
print(f'임베딩 행령 형태 : {embedding_matrix.shape}')
print(f'패딩(id=0)의 임베딩 {embedding_matrix[0]}')
print(f'단어(id=5)의 임베딩 {embedding_matrix[5]}')

임베딩 행령 형태 : (1000, 8)
패딩(id=0)의 임베딩 [0. 0. 0. 0. 0. 0. 0. 0.]
단어(id=5)의 임베딩 [ 0.42097154 -1.3240429   1.4650896   0.05463006 -0.4246347   0.2993056
 -0.7474072  -0.04265463]


In [ ]:
# Rnn 적용
class RnnModule(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(RnnModule, self).__init__()
        self.embdding = nn.Embedding(vocab_size,embedding_dim,padding_idx=0)
        self.rnn  = nn.RNN(embedding_dim,hidden_dim,batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x_emb =  self.embdding(x)  #(batch,seq_len, embedding_dim)
        rnn_out, h_n = self.rnn(x_emb) # rnn_out(batch,seq_len,hidden_dim)
                                     # h_n  (1, batch,hidden_dim)
        # 마지막 스텝의 출력
        last_output = rnn_out[:,-1,:]   # (batch,hidden_dim)
        output = self.sigmoid(self.fc(last_output))
        return output